<a href="https://colab.research.google.com/github/mongoq/thesis/blob/main/Youtube_Yolo_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source: https://towardsdatascience.com/implementing-real-time-object-detection-system-using-pytorch-and-opencv-70bac41148f7

In [1]:
! pip install pafy youtube-dl==2020.12.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 8.0 MB/s 


In [2]:
import cv2
import pafy
import youtube_dl
import torch
from time import time
#import numpy 
import numpy as np
from torch import hub
import requests

In [7]:
#https://stackoverflow.com/questions/63325908/how-do-i-check-if-a-youtube-video-url-is-valid-or-not-in-python

#@title Select Youtube Video { run: "auto" }

video = "Manta Manta Ending" #@param ["Manta Manta Ending", "Own Video"]

if video == "Manta Manta Ending":
  manta_manta = "https://www.youtube.com/watch?v=2sGTPMTaEcA" #@param {type:"string"}
elif video ==  "Own Video":
  own_video = "" #@param {type:"string"}

if video == "Manta Manta Ending":
  url=manta_manta
  print('You selected:', video, "with url:", url)
elif  len(own_video) > 0 and video == "Own Video":
  try:
    r = requests.get(own_video)
    if ("Video unavailable" in r.text):
      print("Not a valid Youtube url!")
    else:
      print('You selected:', video, "with url:", own_video)
      url=own_video
  except:
    print("Please specify (valid) url!")
else:
  print("Please specify url!")

You selected: Manta Manta Ending with url: https://www.youtube.com/watch?v=2sGTPMTaEcA


In [8]:
class Detect():
      #  Read this for classs magic:
      #  https://cito.github.io/byte_of_python/read/class-and-object-vars.html

      # Define Youtube videos 
      #_URL = "https://www.youtube.com/watch?v=wqctLW0Hb_0" # motorway traffic
      #_URL = "https://www.youtube.com/watch?v=2sGTPMTaEcA" # mein schöner benz / manta manta
      _URL = url
      _video_file = "result.mp4" # output file

        #"""
        #Load Yolo V5 Model and init the object
        #"""
      def __init__(self):
        self.modeltorch = torch.hub.load( \
                      'ultralytics/yolov5', \
                      'yolov5s', \
                      pretrained=True)
        #pass

      # Define Video stream
      def get_video_stream(self):
        play = pafy.new(self._URL).streams[-1] #'-1' means read the lowest quality of video.
        assert play is not None # we want to make sure there is a input to read.
        stream = cv2.VideoCapture(play.url) #create a opencv video stream.
        return stream

        """
        The function below identifies the device which is availabe to make the prediction and uses it to load and infer the frame. Once it has results it will extract the labels and cordinates(Along with scores) for each object detected in the frame.
        """
      #def score_frame(frame, model):
      def score_frame(self, frame):
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        #print("Running on: " + str(device))
        #print("score_frame: modeltorch: " + str(type(self.modeltorch)))
        self.modeltorch.to(device)
        results = self.modeltorch(frame)
        #results = np.array(self.modeltorch(frame))
        #results = self.modeltorch(frame).numpy()
        #print("score_frame self.modeltorch(frame): " + str(self.modeltorch(frame)) + " Type: " + str(type(self.modeltorch(frame))))
        #helper = torch.tensor(int(self.modeltorch(frame))).long()
        #helper = torch.tensor(self.modeltorch(frame))
        #print("score_frame helper: " + str(helper))
        #results = helper.to('cpu').numpy()
        #print("results Typ: " + str(type(results)))

        #results.print() - show detected images ... (works!)
        labels = results.xyxyn[0][:, -1].cpu().numpy()
        cord = results.xyxyn[0][:, :-1].cpu().numpy()
        return labels, cord

        """
        The function below takes the results and the frame as input and plots boxes over all the objects which have a score higer than our threshold.
        """
      def plot_boxes(self, results, frame):
        labels, cord = results
        n = len(labels)
        print("Objects detected: " + str(n))
        x_shape, y_shape = frame.shape[1], frame.shape[0]
        for i in range(n):
          row = cord[i]
          # If score is less than 0.2 we avoid making a prediction.
          if row[4] < 0.2: 
            continue
          x1 = int(row[0]*x_shape)
          y1 = int(row[1]*y_shape)
          x2 = int(row[2]*x_shape)
          y2 = int(row[3]*y_shape)
          bgr = (0, 255, 0) # color of the box
          classes = self.modeltorch.names # Get the name of label index
          label_font = cv2.FONT_HERSHEY_SIMPLEX #Font for the label.
          cv2.rectangle(frame, \
                      (x1, y1), (x2, y2), \
                       bgr, 2) #Plot the boxes
          cv2.putText(frame,\
                    classes[labels[i]], \
                    (x1, y1), \
                    label_font, 0.9, bgr, 2) #Put a label over box.
        return frame

        """
        The Function below oracestrates the entire operation and performs the real-time parsing for video stream.
        """
      def __call__(self):
        out_file= self._video_file # Define output file
        player = self.get_video_stream() #Get your video stream.
        assert player.isOpened() # Make sure that their is a stream. 
        #Below code creates a new video writer object to write our
        #output stream.
        x_shape = int(player.get(cv2.CAP_PROP_FRAME_WIDTH))
        y_shape = int(player.get(cv2.CAP_PROP_FRAME_HEIGHT))
        four_cc = cv2.VideoWriter_fourcc(*"MJPG") #Using MJPEG codex
        out = cv2.VideoWriter(out_file, four_cc, 20, \
                          (x_shape, y_shape)) 
        ret, frame = player.read() # Read the first frame.
        while ret: # Run until stream is out of frames
          start_time = time() # We would like to measure the FPS.
          results = self.score_frame(frame) # Score the Frame
          frame = self.plot_boxes(results, frame) # Plot the boxes.
          end_time = time()
          fps = 1/np.round(end_time - start_time, 3) #Measure the FPS.
          #print(f"FPS : {fps}")
          print("FPS: " + str(fps.round(2)))
          out.write(frame) # Write the frame onto the output.
          ret, frame = player.read() # Read next frame.
        print("finished!")

In [ ]:
result=Detect()
result()

In [10]:
from google.colab import files
files.download(Detect._video_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>